In [1]:
import math
import numpy as np
from numpy.random import random
import scipy as sp
from scipy import special
from __future__ import division

def corr(x, y, reps=10**4, rs=None):
    '''
    Simulate permutation p-value for Spearman correlation coefficient
    Returns test statistic, simulations, left-sided p-value, right-sided p-value, two-sided p-value
    '''
    if rs == None:
        rs = np.random.RandomState()
    t = np.corrcoef(x, y)[0,1]
    sims = [np.corrcoef(rs.permutation(x), y)[0,1] for i in range(reps)]
    return t, np.sum(sims <= t)/reps, np.sum(sims >= t)/reps, np.sum(np.abs(sims) >= math.fabs(t))/reps, sims

def stratCorrTst(x, y, group):
    '''
    Calculates sum of Spearman correlations between x and y, computed separately in each group.
    '''
    tst = 0.0
    for g in np.unique(group):
        gg = group == g
        tst += np.corrcoef(x[gg], y[gg])[0,1]
    return tst

def permuteWithinGroups(x, group, rs=None):
    '''
    Permutes the elements of x within groups
    Input: ndarray x to be permuted, ndarray group of group ids, np.random.RandomState object rs
    '''
    if rs == None:
        rs = np.random.RandomState()
    permuted = x.copy()
    for g in np.unique(group):
        gg = group == g
        permuted[gg] = rs.permutation(permuted[gg])      
    return permuted

def stratCorr(x, y, group, rs, reps=10**4):
    '''
    Simulate permutation p-value of stratified Spearman correlation test.
    Returns test statistic, simulations, left-sided p-value, right-sided p-value, two-sided p-value
    '''
    t = stratCorrTst(x, y, group)
    sims = [stratCorrTst(permuteWithinGroups(x, group, rs), y, group) for i in range(reps)]
    return t, np.sum(sims <= t)/reps, np.sum(sims >= t)/reps, np.sum(np.abs(sims) >= math.fabs(t))/reps, sims


In [2]:
seed = 1
rs = np.random.RandomState(seed=seed)
group = np.array([0, 0, 0, 1, 1, 1])
x = np.array([0, 0, 1, 0, 1, 1])
y = np.array([0, 0, 1, 0, 1, 1])
reps = 10**4

In [4]:
## test stratCorr
t, left, right, both, sims = stratCorr(x, y, group, reps=reps, rs=rs)
print t, left, right, both, 1/9 # for the test data, true right p-value is 1/9

2.0 1.0 0.111 0.111 0.111111111111


In [5]:
# test corr
t, left, right, both, sims = corr(x, y, reps=reps, rs=rs)
print t, left, right, both, 1/sp.special.binom(6,3) # for the test data, true right p-value is 1/20

1.0 1.0 0.0549 0.1054 0.05
